In [ ]:
import pandas as pd
from emutools.tex import StandardTexDoc
import arviz as az
from autumn.infrastructure.remote import springboard
from inputs.constants import SUPPLEMENT_PATH
from aust_covid.calibration import get_targets
import plotly.graph_objects as go
from emutools.calibration import round_sigfig, plot_spaghetti, plot_param_hover_spaghetti
import plotly.express as px
from aust_covid.plotting import plot_cdr_examples, plot_subvariant_props
pd.options.plotting.backend = 'plotly'

In [ ]:
rts = springboard.task.RemoteTaskStore()
rts.cd('projects/aust_covid/base_case_analysis')
mt = rts.get_managed_task('2023-09-13T1344-proper_run_with_spaghetti_DEM')
mt.download_all()

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')
targets = get_targets(app_doc)
spaghetti = pd.read_hdf(mt.local.path / 'output/results.hdf', 's')

In [ ]:
indicators = ['notifications_ma', 'deaths_ma', 'adult_seropos_prop', 'reproduction_number']
fig = plot_spaghetti(spaghetti, indicators, 2, targets)
fig

In [ ]:
filename = 'key_outputs.jpg'
fig.write_image(SUPPLEMENT_PATH / filename)
caption = 'Key results for randomly sampled runs from calibration algorithm.'
app_doc.include_figure(caption, filename, 'Results')

In [ ]:
idata = az.from_netcdf(mt.local.path / 'output/calibration_out.nc')
plot_param_hover_spaghetti(spaghetti['notifications_ma'], idata)

In [ ]:
sampled_idata = az.extract(idata, num_samples=10)
fig = plot_cdr_examples(sampled_idata.variables['contact_rate'])
filename = 'cdr_samples.jpg'
fig.write_image(SUPPLEMENT_PATH / filename)
caption = 'Examples of simulated case detection rates over modelled time.'
app_doc.include_figure(caption, filename, 'Outputs', subsection='Notifications')

In [ ]:
plot_subvariant_props(spaghetti)

In [ ]:
filename = 'subvariant_props.jpg'
fig.write_image(SUPPLEMENT_PATH / filename)
caption = 'Proportion of modelled cases attributable to each subvariant over time. ' \
    'Key dates for each variant shown as vertical bars: blue, BA.1; red, BA.2; green, BA.5; ' \
    'dotted, first detection; dashed, \>1\%; solid, \>50\%. '
app_doc.include_figure(caption, filename, 'Results')